In [40]:
import torch
import torch.nn as nn
import numpy as np 
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [41]:
bc = datasets.load_breast_cancer()
#print(bc.DESCR) # description of datasets
print('These are the feature names \n',bc.feature_names)

print('These are target names \n',bc.target_names)
print('These are bc dataset keys \n',bc.keys())

These are the feature names 
 ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
These are target names 
 ['malignant' 'benign']
These are bc dataset keys 
 dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [42]:
X,y = bc.data, bc.target

print(f'Size of "X" is {X.shape}') 
print(f'Size of "y" is {y.shape}')

Size of "X" is (569, 30)
Size of "y" is (569,)


In [43]:
n_samples, n_features_in_X = X.shape
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1212)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(455, 30) (114, 30) (455,) (114,)


### <span style = 'color:cyan'>One of the recommandation to do in logistic regression is scaling down the features into around ***zero means*** and unit variance.

In [44]:
# scaling the features
sc = StandardScaler() 
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [45]:
#convert to tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

print(X_train.shape)
print(y_train.shape)
print(f'y_train is dimension of {y_train.dim()}')

torch.Size([455, 30])
torch.Size([455])
y_train is dimension of 1


## *y_train and y_test has to be converted into two dimensional tensor*

In [46]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

print(f'Now, the y_train has {y_train.dim()} dimension with shape of {y_train.shape}')

Now, the y_train has 2 dimension with shape of torch.Size([455, 1])


# f = sigmoid( $w_{1}x_{1}+w_{2}x_{2}+...+w_{n}x_{n}+b$ )
where n is number of features of one input in a sample training dataset.



## Whenever we create a model class, <span style = 'color:cyan'>forward method has to be implemented</span> so that when we call model ( as in [11] ) with the X_Train input, forward method will automatically calculated and return the predictions.

In [47]:
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features,n_output_features):
        super(LogisticRegression,self).__init__()
        self.linear = nn.Linear(n_input_features,n_output_features)
    
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [48]:
# number of output feature is just one elements from target names.
# That's why n_output_features is one.
learning_rate = 0.01    
num_epochs = 100
model = LogisticRegression(n_input_features = n_features_in_X, n_output_features = 1)

# parameters contains 30 features and a bias. as shown in above equation.
for i in model.parameters():
    print(i)

Parameter containing:
tensor([[ 0.1628, -0.0222, -0.0936,  0.1364,  0.0092, -0.1082,  0.0979, -0.0798,
          0.0214, -0.0205,  0.0337,  0.1451, -0.0889, -0.1425, -0.0605,  0.1439,
          0.0345, -0.1046,  0.0312, -0.0105, -0.1031,  0.0885, -0.0409, -0.0297,
          0.1754,  0.0812,  0.1216,  0.0406,  0.0666, -0.1464]],
       requires_grad=True)
Parameter containing:
tensor([-0.0742], requires_grad=True)


In [49]:
# Binary Cross Entropy loss
criterion = nn.BCELoss()
#stochatic gradient decent
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

In [50]:
for epoch in range(num_epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch %10 == 0:
        print(f'In epoch {epoch+1}, loss is {loss.item():.3f}')
        


In epoch 1, loss is 0.774
In epoch 11, loss is 0.598
In epoch 21, loss is 0.496
In epoch 31, loss is 0.431
In epoch 41, loss is 0.386
In epoch 51, loss is 0.352
In epoch 61, loss is 0.326
In epoch 71, loss is 0.306
In epoch 81, loss is 0.289
In epoch 91, loss is 0.274


In [51]:
with torch.no_grad():
    y_pred_on_X_test = model(X_test)
    print(y_pred_on_X_test[0:5])

tensor([[0.8773],
        [0.7952],
        [0.8615],
        [0.6695],
        [0.8256]])


# <span style='color:cyan'>As we can see, model give us floating values. So, we have to add threshold.</span>
## For example, in this case, we will round the output number. So, the threshold would be 0.5 

In [52]:
y_pred_on_X_test = y_pred_on_X_test.round()


tensor(0.9561)


y_pred_on_X_test.eq(y_test) will give True if an element at *n* index from one tensor is equal to an element of correspoding index of another tensor.

Then, all the True values will be summed up and divided by the number of samples in test tensor.

In [ ]:
accuracy = y_pred_on_X_test.eq(y_test).sum()/float(y_test.shape[0])
print(accuracy)